# Swarm Orchestration with AG2 


````{=mdx}
:::info Requirements
Install `ag2`:
```bash
pip install -U ag2[openai]
```

> **Note:** If you have been using `autogen` or `pyautogen`, all you need to do is upgrade it using:  
> ```bash
> pip install -U autogen
> ```
> or  
> ```bash
> pip install -U pyautogen
> ```
> as `pyautogen`, `autogen`, and `ag2` are aliases for the same PyPI package.  


For more information, please refer to the [installation guide](/docs/user-guide/basic-concepts/installing-ag2).
:::
````

## Set your API Endpoint

The [`config_list_from_json`](https://docs.ag2.ai/docs/api-reference/autogen/config_list_from_json#config-list-from-json) function loads a list of configurations from an environment variable or a json file.

In [3]:
import autogen
import nest_asyncio

nest_asyncio.apply()

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location="/Users/alec/code/ag2/",
    filter_dict={
        "model": ["gpt-4o-mini"],
    },
)

llm_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 1,
    "config_list": config_list,
    "timeout": 120,
    "tools": [],
}

In [6]:
from typing import Annotated
from autogen.agents.experimental.func_agent.func_agent import FuncAgent
from autogen.tools.dependency_injection import Field as AG2Field


def submit_plan(context_variables, plan_steps: Annotated[list[str], AG2Field(description="A list of strings where each string is a step in the plan.")]):
    return plan_steps, str(plan_steps)

def grade_step(context_variables, grade: Annotated[int, AG2Field(description="A grade for the quality of the step in the plan.")]):
    return grade, str(grade)

planner = FuncAgent(name="Planner", runner_llm_config=llm_config,
                                validator_llm_config=llm_config, func_or_tool=submit_plan, 
                                agent_system_message="""
You are a planner agent responsible for taking input from a user and submitting a plan of action to answer it which includes at minimum 3 steps.""",
                                validator_system_message="""
Validations:
1. The plan has at least 3 steps
""")

grader= FuncAgent(name="Grader", runner_llm_config=llm_config,
                                validator_llm_config=llm_config, func_or_tool=grade_step, 
                                agent_system_message="""
You are a grader agent which will grade the quality of a step in a plan.
Rubric: Relevance of a Step in a Plan

Goal: Assess how well a step contributes to the plan's overall objective.

Scale: 0-100 (0 = Irrelevant, 100 = Perfectly Relevant)

Criteria (Key Questions):

Directness: How directly does this step help achieve the plan's goal?
Necessity: Is this step essential, helpful, or unnecessary?
Impact: How much does this step contribute to the overall outcome?
Grading Intervals (Detailed Guidance):

0-20 (Irrelevant): The step has no meaningful connection to the plan's goal. It is either completely unrelated, counterproductive, or nonsensical in the context of the plan.
30-40 (Low Relevance): The step has a weak or indirect connection to the goal. Its contribution is minimal and questionable. It might be tangentially related, but other steps would be far more effective.
50-60 (Moderate Relevance): The step is somewhat relevant and contributes to the goal, but it's not essential. It's a supporting step rather than a driving force. The plan could likely succeed without it.
70-80 (High Relevance): The step is highly relevant and makes a substantial contribution to the goal. It's an important part of the plan and significantly increases the likelihood of success.
90-100 (Perfectly Relevant): The step is essential and highly impactful. It is one of the most critical steps, directly driving the goal. Without this step, the goal is likely unachievable.
Important Instruction for the LLM:

Use any integer from 0 to 100. While the above ranges provide guidance, do not feel limited to only using numbers within those ranges or multiples of 10. You are free to use any whole number (integer) within the 0-100 range, such as 43, 78, 12, 99, 5, 67, etc., to accurately reflect your assessment. The ranges are there to help you understand the scale, but your final score can be any number.
""",
                                validator_system_message="""
Validations:
1. Make sure the score is not a multiple of 10.
""")



question = """Create a plan for another agent to solve the following riddle:
A farmer with a wolf, a goat, and a cabbage must cross a river by boat. The boat can carry only the farmer and a single item. If left unattended together, the wolf would eat the goat, or the goat would eat the cabbage. How can they cross the river without anything being eaten?
"""

plan_res = planner.run_func(question).result_data

for step in plan_res:
    grade = grader.run_func(f"Plan Step:\n {step}").result_data
    print(grade)

_User (to chat_manager):

Create a plan for another agent to solve the following riddle:
A farmer with a wolf, a goat, and a cabbage must cross a river by boat. The boat can carry only the farmer and a single item. If left unattended together, the wolf would eat the goat, or the goat would eat the cabbage. How can they cross the river without anything being eaten?


--------------------------------------------------------------------------------

Next speaker: Planner-Runner


>>>>>>>> USING AUTO REPLY...
Planner-Runner (to chat_manager):

***** Suggested tool call (call_rmZkRvCLH1naDgWEkiyPE3mr): submit_plan *****
Arguments: 
{"plan_steps":["First, the farmer takes the goat across the river and leaves it on the other side.","Next, the farmer returns alone to the original side and takes the cabbage across the river.","The farmer leaves the cabbage on the other side but takes the goat back with him to the original side. He then leaves the goat and takes the wolf across the river."],"hyp

/Users/alec/code/ag2/.venv/lib/python3.11/site-packages/pydantic/main.py:477: UserWarning: Pydantic serializer warnings:
  Expected `list[any]` but got `tuple` with value `()` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_json(


Planner-Validator (to chat_manager):

Validation Result: Passed
Justification: The plan has 3 steps: 1) The farmer takes the goat across the river. 2) The farmer takes the cabbage across the river after returning alone. 3) The farmer takes the wolf across the river after returning with the goat. Therefore, it meets the requirement of having at least 3 steps.

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (859fea54-bb27-4ec4-bee3-d9f35c1b9fbf): No next speaker selected
_User (to chat_manager):

Plan Step:
 First, the farmer takes the goat across the river and leaves it on the other side.

--------------------------------------------------------------------------------

Next speaker: Grader-Runner


>>>>>>>> USING AUTO REPLY...
Grader-Runner (to chat_manager):

***** Suggested tool call (call_oP285YhXUXXfXu5VBoZTip5w): grade_step *****
Arguments: 
{"grade":90,"hypothesis":"This step is essential and highly impactful for achievi